In [2]:
# Install packages
!pip install -r requirements.txt >/dev/null
!pip install eep153_tools >/dev/null
!pip install gnupg 
!pip install gspread_pandas >/dev/null

# Import libraries
import numpy as np
import pandas as pd
import warnings
import fooddatacentral as fdc
from  scipy.optimize import linprog as lp
import plotly.express as px
from eep153_tools.sheets import read_sheets

# API Key
apikey = "Y7y8evhwYYL589qVEZzdHBCyKGeBkNdDWQfnWyIV"



  Using cached gnupg-2.3.1-py3-none-any.whl


In [94]:
# Read diet minimum data
diet_min = pd.read_csv("Spreadsheets/diet_min.csv",index_col=0)
diet_min

,Source,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,,
Energy,---,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,RDA,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",---,14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",RDA,150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",RDA,700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0
"Carbohydrate, by difference",RDA,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0
"Iron, Fe",RDA,7.0,10.0,10.0,8.0,8.0,15.0,11.0,18.0,8.0,18.0,8.0,8.0,8.0
"Magnesium, Mg",RDA,80.0,130.0,130.0,240.0,240.0,360.0,410.0,310.0,400.0,320.0,420.0,320.0,420.0
Niacin,RDA,6.0,8.0,8.0,12.0,12.0,14.0,16.0,14.0,16.0,14.0,16.0,14.0,16.0


In [95]:
# Read diet maximum data
diet_max = pd.read_csv("Spreadsheets/diet_max.csv",index_col=0)
diet_max

,Source,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,,
"Sodium, Na",UL,1500,1900,1900,2200,2200,2300,2300,2300,2300,2300,2300,2300,2300
Energy,NaN,2500,2500,2500,2800,3000,3100,3100,3100,3100,3100,3100,3100,3100


In [6]:
def dietary_ref_intake(age,sex,df):
    """Takes in age and sex, and returns the dietary reference intake for the chosen population"""

    if age <= 3:
        col = 'C 1-3'
    age_ranges = [(4,8),(9,13),(14,18),(19,30),(31,50),(50,100)]
    for age_range in age_ranges:
        if age >= age_range[0] and age <= age_range[1]:
            col = sex + ' ' + str(age_range[0]) + '-' + str(age_range[1])
    return pd.Series(df[col])  

In [7]:
# Example of minimum dietary requirements for a male aged 22
dietary_ref_intake(age=22,sex='M',df=diet_min)

Nutrition
Energy                            2400.0
Protein                             56.0
Fiber, total dietary                33.6
Folate, DFE                        400.0
Calcium, Ca                       1000.0
Carbohydrate, by difference        130.0
Iron, Fe                             8.0
Magnesium, Mg                      400.0
Niacin                              16.0
Phosphorus, P                      700.0
Potassium, K                      4700.0
Riboflavin                           1.3
Thiamin                              1.2
Vitamin A, RAE                     900.0
Vitamin B-12                         2.4
Vitamin B-6                          1.3
Vitamin C, total ascorbic acid      90.0
Vitamin E (alpha-tocopherol)        15.0
Vitamin K (phylloquinone)          120.0
Zinc, Zn                            11.0
Name: M 19-30, dtype: float64

In [8]:
# Example of maximum dietary requirements for a male aged 22
dietary_ref_intake(age=22,sex='M',df=diet_max)

Nutrition
Sodium, Na    2300
Energy        3100
Name: M 19-30, dtype: int64

## Identifying and Uploading FDC

In [96]:
df = pd.read_csv('Spreadsheets/grocery_names.csv')
df['Food'] = df['Item']
df

,Item,Food
0,Hass Avocado,Hass Avocado
1,Tofu,Tofu
2,Lacinato Kale,Lacinato Kale
3,Oat Milk,Oat Milk
4,Flat Leaf Parsley,Flat Leaf Parsley
5,Maradol Papayas,Maradol Papayas
6,Gemelli Pasta,Gemelli Pasta
7,Roma Tomatoes,Roma Tomatoes
8,Serrano Peppers,Serrano Peppers
9,Cilantro,Cilantro


In [92]:
# (IGNORE FOR PROJECT) idea for further automation/ease here: if you inputted a dataframe with your 
# chosen foods having already designated for the "foodCategory" column, that way you could have some 
# kind of if/else statement that filters to find the FIRST index row with that DESIRED (and more accurate) 
# food category as the value in that column of the FDC dataframe. This way you could weed out rando ones, 
# like when we get "Avocado Chunks" as the SECOND result in a search for Hass Avocadoes. 

import fooddatacentral as fdc
import warnings

apikey = 'Y7y8evhwYYL589qVEZzdHBCyKGeBkNdDWQfnWyIV'

def find_FDC_index(API_key, food_list, num_rows): 
    
    fdc_index_results = pd.DataFrame(columns=['Food', 'fdcId', 'description', 'foodCategory'])
    
    for food in food_list: 
        df_food = fdc.search(apikey, food)
        df_food_reduced = df_food[['fdcId', 'description', 'foodCategory']]
        df_food_reduced['Food'] = str(food)
        df_food_4_results = df_food_reduced.iloc[:num_rows]
        fdc_index_results = fdc_index_results.append(df_food_4_results, ignore_index = True)
        
    return fdc_index_results 

In [93]:
get_fdcIds = find_FDC_index(apikey, df['Item'], 1)
get_fdcIds

/tmp/ipykernel_82/3561763109.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_food_reduced['Food'] = str(food)


,Food,fdcId,description,foodCategory
0,Hass Avocado,2609822,"HASS AVOCADOS, HASS",Pre-Packaged Fruit & Vegetables
1,Tofu,2294522,TOFU,Other Meats
2,Lacinato Kale,2495872,"LACINATO TUSCAN KALE CUT SUPER GREENS, LACINAT...",Pre-Packaged Fruit & Vegetables
3,Oat Milk,2257046,"Oat milk, unsweetened, plain, refrigerated",Beverages
4,Flat Leaf Parsley,170416,"Parsley, fresh",Vegetables and Vegetable Products
5,Maradol Papayas,169926,"Papayas, raw",Fruits and Fruit Juices
6,Gemelli Pasta,2618997,"GEMELLI & SPINACH PASTA SALAD, GEMELLI & SPINACH",Deli Salads
7,Roma Tomatoes,1608947,ROMA TOMATOES,Pre-Packaged Fruit & Vegetables
8,Serrano Peppers,2390015,SERRANO PEPPERS,"Pickles, Olives, Peppers & Relishes"
9,Cilantro,399147,CILANTRO,Herbs & Spices


In [82]:
fdcId_array = get_fdcIds['fdcId']
print(fdcId_array)

0     2609822
1     2294522
2     2495872
3     2257046
4      170416
5      169926
6     2618997
7     1608947
8     2390015
9      399147
10    1850647
11    2341591
12    2341550
13    2630019
14    1545857
15     578523
16     173945
17     576920
18     170050
19    1909132
20    2679762
21    1635045
22    2431121
23    2671058
24    2090362
25    1942314
26    1663044
27    2345315
Name: fdcId, dtype: object


In [83]:
file_path = "fdc_ids.csv"

# Save the DataFrame to a CSV file
fdcId_array.to_csv(file_path, index=False)

### Notes from FDC ID DataFrame: 
- for Gemelli Pasta, pd.iloc[0] will get fdcId = 2618997, change to fdcId = 1124597
- "pressed Juices" are fucked up, need to do that search separately
- "Bread" is outputting some cookies and biscuits shit, replace with fdcId = 1913550

In [91]:
gemelli_fdcID = 1124597
bread_fdcID = 1913550
pressedjuice_fdcID = 2095092

In [90]:
# getting FDC Codes

import fooddatacentral as fdc
import warnings
df1 = fdc.search(apikey, 'Cold Pressed Juice')
df1

,fdcId,description,dataType,gtinUpc,publishedDate,brandOwner,brandName,ingredients,marketCountry,foodCategory,...,foodMeasures,foodAttributes,foodAttributeTypes,foodVersionIds,householdServingFullText,shortDescription,commonNames,additionalDescriptions,ndbNumber,subbrandName
0,2075597,COLD PRESSED JUICE,Branded,852043003810,2021-10-28,"The Hain Celestial Group, Inc.",BLUEPRINT,"JUICE FROM: ORGANIC ROMAINE, ORGANIC APPLE, OR...",United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN
1,522238,COLD PRESSED JUICE,Branded,857134006749,2019-04-01,WHOLE FOODS MARKET,NaN,"ORANGE, TURMERIC, CURRY POWDER, BALCK PEPPER",United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],[],[],2 OZA,NaN,NaN,NaN,NaN,NaN
2,1919466,COLD PRESSED JUICE,Branded,653341767899,2021-07-29,"A.M.S. Manufacturing, Inc.",JONI JUICE,"APPLE, KALE, MINT, LEMON, GINGER, SPIRULINA, M...",United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN
3,2095092,COLD PRESSED JUICE,Branded,813377021864,2021-10-28,Urban Remedy,365 WHOLE FOODS MARKET,"PINEAPPLE JUICE, LIME JUICE, GINGER JUICE.",United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN
4,1889790,COLD PRESSED JUICE,Branded,854649005960,2021-07-29,Pressery LLC,WHOLE FOODS MARKET,"CARROTS, BEETS, LEMON, GINGER.",United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN
5,523140,COLD PRESSED JUICE,Branded,857134006640,2019-04-01,WHOLE FOODS MARKET,NaN,"APPLE, CELERY, SPINACH, KALE, LEMON, MATCHA PO...",United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],[],[],8 OZA,NaN,NaN,NaN,NaN,NaN
6,1924431,COLD PRESSED JUICE,Branded,813377021932,2021-07-29,Urban Remedy,WHOLE FOODS MARKET,"APPLE JUICE, CUCUMBER JUICE, LEMON JUICE, DAND...",United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN
7,1917389,COLD PRESSED JUICE,Branded,813377021529,2021-07-29,Urban Remedy,WHOLE FOODS MARKET,"CUCUMBER JUICE, CELERY JUICE, CABBAGE JUICE, R...",United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN
8,1917763,COLD PRESSED JUICE,Branded,857134006350,2021-07-29,Pressery LLC,WHOLE FOODS MARKET,"WATER, LEMON, AGAVE, POMEGRANATE.",United States,Soda,...,[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN
9,1918514,COLD PRESSED JUICE,Branded,813377021628,2021-07-29,Urban Remedy,WHOLE FOODS MARKET,"CUCUMBER JUICE, CELERY JUICE, KALE JUICE, LEMO...",United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN


# TO BE CONTINUED: Getting Nutrition Dataframes (now that FDC Ids are set) 